In [0]:
%pip install requests --quiet


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import requests
import pandas as pd

url = "https://restcountries.com/v3.1/all?fields=name,cca3,region,subregion,population,area,languages,capital"
data = requests.get(url).json()

pdf = pd.json_normalize(data)     # flatten JSON
df = spark.createDataFrame(pdf)   # convert to Spark DF

display(df.limit(10))


cca3,capital,region,subregion,area,population,name.common,name.official,name.nativeName.eng.official,name.nativeName.eng.common,languages.eng,name.nativeName.dzo.official,name.nativeName.dzo.common,languages.dzo,name.nativeName.ita.official,name.nativeName.ita.common,languages.ita,languages.cat,name.nativeName.tvl.official,name.nativeName.tvl.common,languages.tvl,name.nativeName.bjz.official,name.nativeName.bjz.common,name.nativeName.spa.official,name.nativeName.spa.common,languages.bjz,languages.spa,name.nativeName.bis.official,name.nativeName.bis.common,name.nativeName.fra.official,name.nativeName.fra.common,languages.bis,languages.fra,name.nativeName.bel.official,name.nativeName.bel.common,name.nativeName.rus.official,name.nativeName.rus.common,languages.bel,languages.rus,name.nativeName.mfe.official,name.nativeName.mfe.common,languages.mfe,name.nativeName.lao.official,name.nativeName.lao.common,languages.lao,name.nativeName.tur.official,name.nativeName.tur.common,languages.tur,name.nativeName.aym.official,name.nativeName.aym.common,name.nativeName.grn.official,name.nativeName.grn.common,name.nativeName.que.official,name.nativeName.que.common,languages.aym,languages.grn,languages.que,name.nativeName.sin.official,name.nativeName.sin.common,name.nativeName.tam.official,name.nativeName.tam.common,languages.sin,languages.tam,name.nativeName.pih.official,name.nativeName.pih.common,languages.pih,name.nativeName.zho.official,name.nativeName.zho.common,languages.zho,name.nativeName.nld.official,name.nativeName.nld.common,name.nativeName.pap.official,name.nativeName.pap.common,languages.nld,languages.pap,name.nativeName.nfr.official,name.nativeName.nfr.common,languages.nfr,name.nativeName.ara.official,name.nativeName.ara.common,languages.ara,name.nativeName.lat.official,name.nativeName.lat.common,languages.lat,name.nativeName.tsn.official,name.nativeName.tsn.common,languages.tsn,name.nativeName.lit.official,name.nativeName.lit.common,languages.lit,name.nativeName.run.official,name.nativeName.run.common,languages.run,name.nativeName.gle.official,name.nativeName.gle.common,languages.gle,name.nativeName.fin.official,name.nativeName.fin.common,name.nativeName.swe.official,name.nativeName.swe.common,languages.fin,languages.swe,name.nativeName.ben.official,name.nativeName.ben.common,languages.ben,name.nativeName.dan.official,name.nativeName.dan.common,languages.dan,name.nativeName.ron.official,name.nativeName.ron.common,languages.ron,name.nativeName.bul.official,name.nativeName.bul.common,languages.bul,name.nativeName.afr.official,name.nativeName.afr.common,name.nativeName.deu.official,name.nativeName.deu.common,name.nativeName.her.official,name.nativeName.her.common,name.nativeName.hgm.official,name.nativeName.hgm.common,name.nativeName.kwn.official,name.nativeName.kwn.common,name.nativeName.loz.official,name.nativeName.loz.common,name.nativeName.ndo.official,name.nativeName.ndo.common,languages.afr,languages.deu,languages.her,languages.hgm,languages.kwn,languages.loz,languages.ndo,name.nativeName.nor.official,name.nativeName.nor.common,languages.nor,name.nativeName.ltz.official,name.nativeName.ltz.common,languages.ltz,name.nativeName.jpn.official,name.nativeName.jpn.common,languages.jpn,name.nativeName.sqi.official,name.nativeName.sqi.common,languages.sqi,name.nativeName.smo.official,name.nativeName.smo.common,name.nativeName.tkl.official,name.nativeName.tkl.common,languages.smo,languages.tkl,name.nativeName.ton.official,name.nativeName.ton.common,languages.ton,name.nativeName.tuk.official,name.nativeName.tuk.common,languages.tuk,name.nativeName.sag.official,name.nativeName.sag.common,languages.sag,name.nativeName.lav.official,name.nativeName.lav.common,languages.lav,name.nativeName.niu.official,name.nativeName.niu.common,languages.niu,name.nativeName.srp.official,name.nativeName.srp.common,languages.srp,name.nativeName.rar.official,name.nativeName.rar.common,languages.rar,name.nativeName.mlg.official,name.nativeName.mlg.common,language

In [0]:
# ===== Step 2: CLEAN & PREPARE (run this cell) =====
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when, size, element_at
from functools import reduce

# `df` is the Spark DataFrame you already created from ingestion
cols = df.columns

# pick name column (pandas.json_normalize often creates 'name_common')
if 'name_common' in cols:
    name_col = 'name_common'
elif 'name.common' in cols:
    name_col = 'name.common'
elif 'name' in cols:
    name_col = 'name'
else:
    raise Exception("No country name column found (looked for name_common / name.common / name)")

# capital column (if exists)
capital_col = 'capital' if 'capital' in cols else None

# languages columns produced by json_normalize often start with 'languages'
lang_cols = [c for c in cols if c.startswith('languages')]

# build languages_count expression (count non-null language columns) — simple and robust
if len(lang_cols) > 0:
    lang_exprs = [ when(col(c).isNotNull(), F.lit(1)).otherwise(F.lit(0)) for c in lang_cols ]
    languages_count_expr = reduce(lambda a,b: a + b, lang_exprs).alias("languages_count")
else:
    languages_count_expr = F.lit(0).alias("languages_count")

# prepare list of select expressions (use safe fallbacks if a column is missing)
select_exprs = [
    col(name_col).alias("country_name"),
    col("cca3").alias("iso3") if "cca3" in cols else F.lit(None).alias("iso3"),
    col("region").alias("region") if "region" in cols else F.lit(None).alias("region"),
    col("subregion").alias("subregion") if "subregion" in cols else F.lit(None).alias("subregion"),
    col("population").cast("long").alias("population") if "population" in cols else F.lit(None).cast("long").alias("population"),
    col("area").cast("double").alias("area") if "area" in cols else F.lit(None).cast("double").alias("area"),
    languages_count_expr
]

# capital: if it's an ARRAY, safely take first element only when size>0; otherwise return as-is
if capital_col:
    capital_safe = when((size(col(capital_col)) > 0), element_at(col(capital_col), 1)).otherwise(None).alias("capital")
    select_exprs.append(capital_safe)
else:
    select_exprs.append(F.lit(None).alias("capital"))

# build cleaned DF
clean_df = df.select(*select_exprs)

# drop rows missing critical fields
clean_df = clean_df.dropna(subset=["country_name", "population"])

# preview
display(clean_df.limit(10))

# save final delta table (uncomment the two lines below if you want to persist now)
clean_df.write.format("delta").mode("overwrite").saveAsTable("countries_final")
print("Saved table: countries_final (rows = {})".format(clean_df.count()))


country_name,iso3,region,subregion,population,area,languages_count,capital
Antigua and Barbuda,ATG,Americas,Caribbean,103603,442.0,1,Saint John's
Bhutan,BTN,Asia,Southern Asia,784043,38394.0,1,Thimphu
Italy,ITA,Europe,Southern Europe,58927633,301336.0,2,Rome
Tuvalu,TUV,Oceania,Polynesia,10643,26.0,2,Funafuti
Anguilla,AIA,Americas,Caribbean,16010,91.0,1,The Valley
Australia,AUS,Oceania,Australia and New Zealand,27536874,7692024.0,1,Canberra
Belize,BLZ,Americas,Central America,417634,22966.0,3,Belmopan
Vanuatu,VUT,Oceania,Melanesia,321409,12189.0,3,Port Vila
Belarus,BLR,Europe,Eastern Europe,9109280,207600.0,2,Minsk
Mauritius,MUS,Africa,Eastern Africa,1243741,2040.0,3,Port Louis


Saved table: countries_final (rows = 250)


In [0]:
# Step 2: save cleaned dataframe to Delta and show quick checks
# (replace clean_df with the variable name you used if different)

# 1) Save as a Delta table named countries_final in the current database
clean_df.write.format("delta").mode("overwrite").saveAsTable("countries_final")

# 2) Show 5 rows to confirm
display(spark.table("countries_final").limit(5))

# 3) Print total rows count
print("Rows in countries_final =", spark.table("countries_final").count())



country_name,iso3,region,subregion,population,area,languages_count,capital
Antigua and Barbuda,ATG,Americas,Caribbean,103603,442.0,1,Saint John's
Bhutan,BTN,Asia,Southern Asia,784043,38394.0,1,Thimphu
Italy,ITA,Europe,Southern Europe,58927633,301336.0,2,Rome
Tuvalu,TUV,Oceania,Polynesia,10643,26.0,2,Funafuti
Anguilla,AIA,Americas,Caribbean,16010,91.0,1,The Valley


Rows in countries_final = 250


In [0]:
# Step 2 — save cleaned table and verify
# (assumes your cleaned DataFrame variable is `clean_df`)

clean_df.write.format("delta").mode("overwrite").saveAsTable("countries_final")

# show a preview and row count to prove it's saved
display(spark.table("countries_final").limit(10))
print("Saved countries_final — rows:", spark.table("countries_final").count())



country_name,iso3,region,subregion,population,area,languages_count,capital
Antigua and Barbuda,ATG,Americas,Caribbean,103603,442.0,1,Saint John's
Bhutan,BTN,Asia,Southern Asia,784043,38394.0,1,Thimphu
Italy,ITA,Europe,Southern Europe,58927633,301336.0,2,Rome
Tuvalu,TUV,Oceania,Polynesia,10643,26.0,2,Funafuti
Anguilla,AIA,Americas,Caribbean,16010,91.0,1,The Valley
Australia,AUS,Oceania,Australia and New Zealand,27536874,7692024.0,1,Canberra
Belize,BLZ,Americas,Central America,417634,22966.0,3,Belmopan
Vanuatu,VUT,Oceania,Melanesia,321409,12189.0,3,Port Vila
Belarus,BLR,Europe,Eastern Europe,9109280,207600.0,2,Minsk
Mauritius,MUS,Africa,Eastern Africa,1243741,2040.0,3,Port Louis


Saved countries_final — rows: 250


In [0]:
spark.sql("""
SELECT country_name, population
FROM countries_final
ORDER BY population DESC
LIMIT 10
""").display()



country_name,population
India,1417492000
China,1408280000
United States,340110988
Indonesia,284438782
Pakistan,241499431
Nigeria,223800000
Brazil,213421037
Bangladesh,169828911
Russia,146028325
Mexico,130575786


In [0]:

spark.sql("""
SELECT region, COUNT(*) AS total_countries
FROM countries_final
GROUP BY region
ORDER BY total_countries DESC
""").display()


region,total_countries
Africa,59
Americas,56
Europe,53
Asia,50
Oceania,27
Antarctic,5


In [0]:
df_final = spark.table("countries_final")
display(df_final)


country_name,iso3,region,subregion,population,area,languages_count,capital
Antigua and Barbuda,ATG,Americas,Caribbean,103603,442.0,1,Saint John's
Bhutan,BTN,Asia,Southern Asia,784043,38394.0,1,Thimphu
Italy,ITA,Europe,Southern Europe,58927633,301336.0,2,Rome
Tuvalu,TUV,Oceania,Polynesia,10643,26.0,2,Funafuti
Anguilla,AIA,Americas,Caribbean,16010,91.0,1,The Valley
Australia,AUS,Oceania,Australia and New Zealand,27536874,7692024.0,1,Canberra
Belize,BLZ,Americas,Central America,417634,22966.0,3,Belmopan
Vanuatu,VUT,Oceania,Melanesia,321409,12189.0,3,Port Vila
Belarus,BLR,Europe,Eastern Europe,9109280,207600.0,2,Minsk
Mauritius,MUS,Africa,Eastern Africa,1243741,2040.0,3,Port Louis


In [0]:
df_final.printSchema()


root
 |-- country_name: string (nullable = true)
 |-- iso3: string (nullable = true)
 |-- region: string (nullable = true)
 |-- subregion: string (nullable = true)
 |-- population: long (nullable = true)
 |-- area: double (nullable = true)
 |-- languages_count: integer (nullable = true)
 |-- capital: string (nullable = true)



In [0]:
df_final.describe(["population", "area"]).show()


+-------+-------------------+------------------+
|summary|         population|              area|
+-------+-------------------+------------------+
|  count|                250|               250|
|   mean|      3.207798184E7|      601038.85084|
| stddev|1.319655234265103E8|1912575.3624724469|
|    min|                  0|              0.49|
|    max|         1417492000|       1.7098246E7|
+-------+-------------------+------------------+



In [0]:
df_final.groupBy("region").count().show()


+---------+-----+
|   region|count|
+---------+-----+
| Americas|   56|
|     Asia|   50|
|   Europe|   53|
|  Oceania|   27|
|   Africa|   59|
|Antarctic|    5|
+---------+-----+



In [0]:
df_final.orderBy(df_final.area.desc()).limit(10).show()


+-------------+----+---------+--------------------+----------+-----------+---------------+----------------+
| country_name|iso3|   region|           subregion|population|       area|languages_count|         capital|
+-------------+----+---------+--------------------+----------+-----------+---------------+----------------+
|       Russia| RUS|   Europe|      Eastern Europe| 146028325|1.7098246E7|              1|          Moscow|
|   Antarctica| ATA|Antarctic|                    |      1300|      1.4E7|              0|            NULL|
|       Canada| CAN| Americas|       North America|  41651653|  9984670.0|              2|          Ottawa|
|        China| CHN|     Asia|        Eastern Asia|1408280000|  9706961.0|              1|         Beijing|
|United States| USA| Americas|       North America| 340110988|  9525067.0|              1|Washington, D.C.|
|       Brazil| BRA| Americas|       South America| 213421037|  8515767.0|              1|        Brasília|
|    Australia| AUS|  Oceani

In [0]:
df_final.filter(df_final.region == "Europe").show()


+--------------------+----+------+----------------+----------+-----------+---------------+--------------+
|        country_name|iso3|region|       subregion|population|       area|languages_count|       capital|
+--------------------+----+------+----------------+----------+-----------+---------------+--------------+
|               Italy| ITA|Europe| Southern Europe|  58927633|   301336.0|              2|          Rome|
|             Belarus| BLR|Europe|  Eastern Europe|   9109280|   207600.0|              2|         Minsk|
|            Guernsey| GGY|Europe| Northern Europe|     64781|       78.0|              3|St. Peter Port|
|        Vatican City| VAT|Europe| Southern Europe|       882|       0.49|              2|  Vatican City|
|           Lithuania| LTU|Europe| Northern Europe|   2894886|    65300.0|              1|       Vilnius|
|             Ireland| IRL|Europe| Northern Europe|   5458600|    70273.0|              2|        Dublin|
|             Finland| FIN|Europe| Northern Eu

In [0]:
spark.table("countries_final").columns


['country_name',
 'iso3',
 'region',
 'subregion',
 'population',
 'area',
 'languages_count',
 'capital']

In [0]:
# Simple regression (PySpark ML) — predict population from area
from pyspark.sql.functions import col, log1p
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# 1) Load saved table (use your table name)
df = spark.table("countries_final").select("country_name", "population", "area")

# 2) Keep only rows with numeric area and population > 0
df = df.filter(col("population").isNotNull() & col("area").isNotNull() & (col("area") > 0) & (col("population") > 0))

# 3) Transform target and feature: log1p to reduce skew
df = df.withColumn("label", log1p(col("population"))) \
       .withColumn("log_area", log1p(col("area")))

# 4) Assemble feature vector
assembler = VectorAssembler(inputCols=["log_area"], outputCol="features")
data = assembler.transform(df).select("features", "label")

# 5) Train/test split
train, test = data.randomSplit([0.8, 0.2], seed=42)

# 6) Train simple Linear Regression model
lr = LinearRegression(featuresCol="features", labelCol="label")
model = lr.fit(train)

# 7) Predict on test set
pred = model.transform(test)

# 8) Evaluate (RMSE on log scale and R2)
rmse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse").evaluate(pred)
r2 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2").evaluate(pred)

print(f"Model coefficient: {model.coefficients[0]:.4f}, intercept: {model.intercept:.4f}")
print(f"RMSE (log scale): {rmse:.4f}")
print(f"R2: {r2:.4f}")

# 9) Show some predictions (converted back to original scale for interpretability)
from pyspark.sql.functions import expm1
display(
    pred.select(
        expm1(col("prediction")).alias("pred_population"),
        expm1(col("label")).alias("actual_population")
    ).orderBy(col("actual_population").desc()).limit(10)
)

# Optional: save model (uncomment if you want to persist)
# model.write().overwrite().save("dbfs:/models/country_pop_linear_regression")


Model coefficient: 0.7502, intercept: 6.8989
RMSE (log scale): 1.7437
R2: 0.6767


pred_population,actual_population
1.7037970724287543E8,3.4011098799999976E8
1.0915775167916648E7,6.928143699999999E7
3.648961739433033E7,6.310094500000007E7
1.2773602900435794E7,5.8927633000000015E7
2.34325638976703E7,5.131675599999995E7
3.791913060833338E7,3.435024400000001E7
2.942269263802291E7,2.8516999999999955E7
1.5522913701361254E7,1.7073087000000007E7
1.762495850624921E7,1.1781558999999998E7
4881940.75195158,1.129424300000001E7


In [0]:
# ----- 1. LOAD DATA -----
df = spark.table("countries_final")

# Keep only numeric columns needed for classification
# population, area, languages_count already exist from your previous steps
data = df.select("population", "area", "languages_count")

# ----- 2. CREATE A BINARY TARGET LABEL -----
# Label = 1 if population > median, else 0
median_pop = data.approxQuantile("population", [0.5], 0.0)[0]
data = data.withColumn("label", (data.population > median_pop).cast("int"))

# ----- 3. PREPARE FEATURES -----
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(
    inputCols=["population", "area", "languages_count"],
    outputCol="features"
)
final_data = assembler.transform(data).select("features", "label")

# ----- 4. TRAIN / TEST SPLIT -----
train, test = final_data.randomSplit([0.7, 0.3], seed=42)

# ----- 5. TRAIN CLASSIFICATION MODEL (Logistic Regression) -----
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol="features", labelCol="label")
model = lr.fit(train)

# ----- 6. PREDICT -----
predictions = model.transform(test)

# ----- 7. EVALUATE ACCURACY -----
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print("Classification Model Accuracy =", accuracy)
predictions.select("features", "label", "prediction").show(10, truncate=False)


Classification Model Accuracy = 0.9846153846153847
+--------------------+-----+----------+
|features            |label|prediction|
+--------------------+-----+----------+
|[0.0,60.0,1.0]      |0    |0.0       |
|[400.0,7747.0,1.0]  |0    |0.0       |
|[882.0,0.49,2.0]    |0    |0.0       |
|[1300.0,1.4E7,0.0]  |0    |0.0       |
|[2530.0,61399.0,1.0]|0    |0.0       |
|[2608.0,12.0,3.0]   |0    |0.0       |
|[3662.0,12173.0,1.0]|0    |0.0       |
|[10562.0,21.0,1.0]  |0    |0.0       |
|[11620.0,142.0,1.0] |0    |0.0       |
|[15040.0,236.0,2.0] |0    |0.0       |
+--------------------+-----+----------+
only showing top 10 rows


In [0]:
# ============================
# SIMPLE FORECASTING MODEL
# Predicting 'population' from 'area'
# ============================

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.sql.functions import col

# Load the cleaned table
df = spark.table("countries_final")

# Keep only rows with numeric values
df_clean = df.select("area", "population") \
             .filter(col("area").isNotNull()) \
             .filter(col("population").isNotNull())

# Assemble feature column
assembler = VectorAssembler(
    inputCols=["area"],
    outputCol="features"
)

df_features = assembler.transform(df_clean)

# Train-test split
train, test = df_features.randomSplit([0.8, 0.2], seed=42)

# Create forecasting (regression) model
lr = LinearRegression(
    featuresCol="features",
    labelCol="population"
)

# Train model
lr_model = lr.fit(train)

# Model summary
print("Intercept:", lr_model.intercept)
print("Coefficient:", lr_model.coefficients)

# Predict (Forecast)
predictions = lr_model.transform(test)

# Show predictions
predictions.select("area", "population", "prediction").show(15, truncate=False)


Intercept: 17346027.677642353
Coefficient: [27.62509461563237]
+------+----------+--------------------+
|area  |population|prediction          |
+------+----------+--------------------+
|6.0   |38000     |1.7346193428210046E7|
|21.0  |11680     |1.734660780462928E7 |
|30.0  |685900    |1.7346856430480823E7|
|49.0  |0         |1.7347381307278518E7|
|91.0  |16010     |1.7348541561252374E7|
|142.0 |11620     |1.7349950441077773E7|
|236.0 |15040     |1.7352547199971642E7|
|316.0 |574250    |1.7354757207540892E7|
|444.0 |156115    |1.7358293219651695E7|
|452.0 |122729    |1.735851422040862E7 |
|459.0 |16733     |1.7358707596070927E7|
|468.0 |88406     |1.735895622192247E7 |
|572.0 |84530     |1.7361829231762495E7|
|747.0 |100179    |1.736666362332023E7 |
|1128.0|349925    |1.7377188784368787E7|
+------+----------+--------------------+
only showing top 15 rows
